In [ ]:
column_transformations=[
  {
    "categorical": {
      "columnName": "what_is_your_col_name_1"
    }
  },
  {
    "numeric": {
      "columnName": "what_is_your_col_name_2"
    }
  },
  {
    "auto": {
      "columnName": "your_split_col_need_to_add_toooooo"
    }
  }
]

In [ ]:
import pandas as pd
import time

from google.cloud import aiplatform as vertex_ai
from google.cloud import bigquery

In [ ]:
PREFIX = 'project_name'

shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT = shell_output[0]
print("Project ID: ", PROJECT)

In [ ]:
REGION = 'us-central1'
# STAGING_BUCKET = f'gs://{PREFIX}-bucket'
# VERTEX_SA = f'training-sa@{PROJECT}.iam.gserviceaccount.com'
BQ_DATASET_NAME = 'dataset_name' 
BQ_TABLE_NAME = 'table_name'
BQ_LOCATION = 'US'

In [ ]:
vertex_ai.init(
    project=PROJECT,
    location=REGION
)

In [ ]:
display_name = 'display_name'
bq_source_uri = f'bq://{PROJECT}.{BQ_DATASET_NAME}.{BQ_TABLE_NAME}'

filter = f'display_name="{display_name}"'

dataset = vertex_ai.TabularDataset.list(filter=filter)
if not dataset:
    print("Creating a new dataset.")
    dataset = vertex_ai.TabularDataset.create(
        display_name=display_name, bq_source=bq_source_uri,
    )

    dataset.wait()
else:
    print("Using existing dataset: ", dataset[0].resource_name)
    dataset = vertex_ai.TabularDataset(dataset_name=dataset[0].resource_name)


In [ ]:
model_display_name = 'model_name'
target_column = 'target_col'
optimization_prediction_type = 'classification'
optimization_objective = 'minimize-log-loss'
# classification(binary):maximize-au-roc/minimize-log-loss/maximize-au-prc/maximize-precision-at-recall/maximize-recall-at-precision
# classification(multi-class):minimize-log-loss
# regression:minimize-rmse/minimize-mae/minimize-rmsle
optimization_objective_precision_value = 0.5
split_column = 'your_split_col_need_to_add_toooooo'
weight_column = 'weight_col'
budget_milli_node_hours = 1
disable_early_stopping = True
sync = True


job = vertex_ai.AutoMLTabularTrainingJob(
    display_name=display_name,
    optimization_prediction_type=optimization_prediction_type,
    optimization_objective=optimization_objective,
    optimization_objective_precision_value=optimization_objective_precision_value,
    column_transformations=column_transformations,
)

model = job.run(
    dataset=dataset,
    target_column=target_column,
    budget_milli_node_hours=budget_milli_node_hours,
    model_display_name=model_display_name,
    predefined_split_column_name=split_column,
    weight_column=weight_column,
    disable_early_stopping=disable_early_stopping,
    sync=sync
)